In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:


songsRawData = pd.read_csv('Data\ml-03-data-processing-songs-dataset.csv')


In [ ]:
print(songsRawData.describe())


In [ ]:
songsData = songsRawData.copy()
songsData['Year'] = pd.to_numeric(songsData['Year'])
plt.hist(songsData['Year'])
plt.show()


The Year has outlier data that are likely incorrectly entered. The minimum value of 92 may have been a miss-entry for 1992. There may be other similar outliers to handle. 

In [ ]:
print(songsData[songsData['Year'] < 1900][['Title', 'Artist', 'Year']])


Googling some of the songs on the list indicate that they were either released or rereleased in 1992. The outliers can be replaced with 1992.

In [ ]:
songsData['Year'] = songsData['Year'].replace(92, 1992)
print(songsData.describe())
plt.hist(songsData['Year'])
plt.show()
print(songsData.info())

There is something weird with the Length column in that it is an object.

In [ ]:
print(songsData['Length (Duration)'].unique)
# Check if all are Numeric
for idx, row in songsData.iterrows():
    try: 
        test = pd.to_numeric(row['Length (Duration)'])
    except: 
        print(f"Bad Length: {row['Length (Duration)']}")

# There are commas messing things up in our conversion function
# so, remove the commas and try converting again. (Use to_numeric because of NaNs)
songsData['Length (Duration)'] = songsData['Length (Duration)'].str.replace(',', '')
# Check if all are Numeric
for idx, row in songsData.iterrows():
    try:
        row['Length (Duration)'] = pd.to_numeric(row['Length (Duration)'])
    except:
        print(f"Bad Length: {row['Length (Duration)']}")
# Change the dtype for the column now that all the values are fixed
songsData['Length (Duration)'] = songsData['Length (Duration)'].astype(float)
print(songsData['Length (Duration)'].unique)


2. There are 1994 observations but many of the columns have a lot of missing values. In particular, Daceability and Valence. Title, Artist, Genre, Month, and Length are non-numeric types. Remove all rows that have over half the values missing.

In [ ]:

songsData.dropna(thresh=len(songsData.axes[1])/2, axis=0, inplace=True)
print(songsData.info())
songsData[songsData['Artist'].isna()]


This removed ten rows from the dataframe that were missing most of their values. Do the same thing for the columns.


In [ ]:

songsData.dropna(thresh=len(songsData.axes[0])/2+1, axis=1, inplace=True)
print(songsData.info())


This removed 3 columns

3. Calculate descriptive statistics for each column. Let's define an outlier as a value at least 3 standard deviations away from the mean. Which columns have outliers? What are those values?

In [ ]:
# loop through all of the columns of type float64. Need to do to avoid failure
# when trying to evaluate strings.
for name, values in songsData.select_dtypes('float64').iteritems():
    mean = pd.Series.mean(values)
    stddev = pd.Series.std(values)
    upper = mean + 3*stddev
    lower = mean - 3*stddev
    print(name)
    print(f"Mean/Std Dev: {mean}, {stddev}")
    print(f"Upper/Lower: {upper} to {lower}")
    OutOfBounds = []
    for idx, value in enumerate(values):
        # If we have outliers add them to the list to print
        # Also, do a winsorizing by setting them to the nearest boundary. 
        if not pd.isna(value):
            if (value > upper):
                OutOfBounds.append(value)
                songsData[name].iloc[idx] = upper
            elif (value < lower):
                OutOfBounds.append(value)
                songsData[name].iloc[idx] = lower

    print(f"Outliers: {OutOfBounds}")
print(pd.Series.mean(songsData['Beats Per Minute (BPM)']))
print(pd.Series.std(songsData['Beats Per Minute (BPM)']))




4. With the remaining columns, use scikit-learn to impute missing values. For continuous features, fill in the mean. For categorical features, fill in the mode.

In [ ]:
# Remember what types the columns are
print(songsData.info())

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

## Impute the numerical values to the mean
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
for name, values in songsData.select_dtypes('float64').iteritems():
    songsData[name] = imp_mean.fit_transform(
        songsData[name].to_numpy().reshape(-1, 1))

print(songsData.info())

In [ ]:
## Impute the categorical values to the mode
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
for name, values in songsData.select_dtypes('object').iteritems():
    songsData[name] = imp_mean.fit_transform(
        songsData[name].to_numpy().reshape(-1, 1))

print(songsData.info())


5. Combine the date-related columns into one column with the Pandas to_datetime() method, then use that column to create a numeric Age column (in years). Calculate Age based on today's date; it doesn't have to be a whole number. Once you've created the Age column, remove the other date-related columns, including the one you created with Pandas.

In [ ]:
songsData['Month'].unique()

The month column is string object and contains a mixture of numeric values and short month names. We need to pull out the short month names and convert to numeric.

In [ ]:
import calendar 
MonthAbbr = list(calendar.month_abbr)
for abbr in MonthAbbr:
    songsData['Month'].replace(abbr, MonthAbbr.index(abbr), inplace=True)
songsData['Month'].unique()



In [ ]:
songsData['Month'] = songsData['Month'].astype(str)
songsData['Year'] = songsData['Year'].astype(int).astype(str)
songsData.info()


Now, we can combine the month and year into a datatime object and put in another column.

In [ ]:
songsData['YearMonth'] = pd.to_datetime(songsData['Year'] + ' '+ songsData['Month'])
songsData['YearMonth'].head()

Calculate the age based on the year/month of release and add to an Age column. This is an integer age without fractional values. 

In [ ]:
now = pd.to_datetime('now')
songsData['Age'] = (now - songsData['YearMonth']).astype('<m8[Y]').astype(int)
print(songsData['Age'])


In [ ]:
songsData.info()

Remove the other date columns leaving only age

In [ ]:
songsData.drop(['Year','Month','YearMonth'], axis = 1, inplace=True)
songsData.info()

6. Create dummy variables for the categorical features. Drop one level of each feature to end up with k-1 dummies, not k.

In [ ]:
songsData = pd.get_dummies(songsData, drop_first=True)
print(songsData.shape)
songsData.head()

In [ ]:
# Clean up a bit and export
songsData.drop('Index', axis=1, inplace=True)
songsData.reset_index(drop=True, inplace=True)
songsData.to_csv('Data/M03DataProcessingClean.csv')
